<a href="https://colab.research.google.com/github/caiocesarcosta/transform-image-gray-binary-project2/blob/main/transform_image_gray_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import drive
import os
from IPython.display import Image, display
import io
import pdb

drive.mount('/content/drive', force_remount=True)

class Image:
    def __init__(self, file_manager):
        self.imageWithoutJPG = None
        self.width = None
        self.height = None
        if file_manager:
            self.imageWithoutJPG = file_manager.imageWithoutJPG
            self.width, self.height = file_manager.getImageDimensions()

    def isImage(self, image_bytes):
        return image_bytes is not None

    def printImagem(self, arr, title):
        """Display a NumPy array as a string representation."""
        print(f"\n{title}:\n{arr}\n")


class ImageGrayScale(Image):
    def __init__(self, file_manager):
        super().__init__(file_manager)
        self.imageGray = None
        self.rgb_data = None
        self.weightRed = 0.2989
        self.weightGreen = 0.5870
        self.weightBlue = 0.1140
        if file_manager and self.width and self.height:
            self.rgb_data = self._extractColorDataRGB(self.imageWithoutJPG, self.width, self.height)
            self.imageGray = self._calculateWeightedAverageGrayscale()

    def _extractColorDataRGB(self, image_bytes, width, height):
        """Extrair dados de cor (R, G, B) da imagem."""
        if image_bytes is None:
            return None
        if width is None or height is None:
            print("Dimensões da imagem não encontradas para conversão em tons de cinza")
            return None
        try:
            # Extrair dados de cor (R, G, B) da imagem
            rgb_data = np.frombuffer(image_bytes, dtype=np.uint8).reshape(height, width, 3)
            return rgb_data
        except Exception as e:
            print(f"Erro ao extrair dados de cor RGB: {e}")
            return None

    def _calculateWeightedAverageGrayscale(self):
        """Cálculo da média ponderada que retona gray_data."""
        if self.rgb_data is None:
            return None
        try:
            gray_data = np.dot(self.rgb_data[..., :3], [self.weightRed, self.weightGreen, self.weightBlue]).astype(np.uint8)
            return gray_data
        except Exception as e:
            print(f"Erro na conversão para escala de cinza: {e}")
            return None


class ImageBinary(Image):
    def __init__(self, file_manager, gray_image):
        super().__init__(file_manager)
        self.imageBinary = None
        self.threshold = 128
        if self.isImage(gray_image):
            self.imageBinary = self._binarizeImag(gray_image, self.threshold)

    def _binarizeImag(self, gray_image, threshold):
        """Binariza a imagem em escala de cinza usando um limiar."""
        if gray_image is None:
            return None
        try:
            binary_image = (gray_image > threshold).astype(np.uint8) * 255
            return binary_image
        except Exception as e:
            print(f"Erro na binarização: {e}")
            return None


class FileManagement:
    def __init__(self, image_path):
        self.imagePath = image_path
        self.imageBytes = None
        self.imageWithoutJPG = None
        try:
            drive.mount('/content/drive', force_remount=True)
        except Exception as e:
            print(f"Erro ao montar o Google Drive: {e}")
            return None
        self.imageBytes = self.readImageFromDrive()
        if self.imageBytes is not None:
            self.imageWithoutJPG = self._removeExtensionJPG()

    def readImageFromDrive(self):
        """Lê uma imagem do Google Drive e retorna como um array numpy."""
        try:
            if not os.path.exists(self.imagePath):
                print(f"Erro: Arquivo não encontrado: {self.imagePath}")
                return None

            with open(self.imagePath, 'rb') as f:
                image_data = f.read()
            return image_data
        except FileNotFoundError:
            print(f"Erro: arquivo não encontrado: {self.imagePath}")
            return None
        except Exception as e:
            print(f"Erro ao ler a imagem: {e}")
            return None

    def _removeExtensionJPG(self):
        """Remove o header JPG de tamanho variável."""
        image_bytes = self.imageBytes
        if image_bytes is None:
            return None
        try:
            pos = 0
            while pos < len(image_bytes):
              if image_bytes[pos] == 0xFF and image_bytes[pos + 1] == 0xDA:
                image_bytes = image_bytes[pos + 2:]
                return image_bytes
                pos += 1
                # Se o marcador não for encontrado, retorna None
            print("Marcador de header não encontrado")
            return None
        except Exception as e:
           print(f"Erro ao remover o header: {e}")
        return None


    def getImageDimensions(image_bytes):
        """
        Get image dimensions from JPEG header with validation.
        This function provides an alternative method of extracting
        image dimensions, particularly useful if the primary method fails.
        """
        if image_bytes is None:
            return None, None
        try:
            pos = 0
            size = len(image_bytes)
            while pos < size:
                if image_bytes[pos] == 0xFF:
                    if pos + 1 < size and image_bytes[pos + 1] == 0xC0:
                        if image_bytes[pos + 2] == 0x08:
                            height = (image_bytes[pos + 3] << 8) | image_bytes[pos + 4]
                            width = (image_bytes[pos + 5] << 8) | image_bytes[pos + 6]
                            if height > 0 and width > 0:  # Basic validation of dimensions
                                return width, height
                            else:
                                print("Invalid height or width detected.")
                                return None, None
                        else:
                            print("Unexpected JPEG format detected.")
                            return None, None
                pos += 1
            print("SOF0 marker not found in JPEG file.")
            return None, None
        except Exception as e:
            print(f"Erro ao ler dimensões da imagem: {e}")
            return None, None

    def getImageDimensions(self):
        """Tenta determinar as dimensões de uma imagem JPEG a partir de seus bytes."""

        pdb.set_trace()
        image_bytes = self.imageBytes
        if image_bytes is None:
            return None, None
        try:
            pos = 0
            size = len(image_bytes)
            while pos < size:
              if image_bytes[pos] == 0xFF:
                  if pos + 1 < size and image_bytes[pos + 1] == 0xC0:
                      # Encontrou o marcador SOF0, agora verifique se o byte seguinte indica o formato correto
                      if image_bytes[pos + 2] == 0x11:
                          height = (image_bytes[pos + 5] << 8) | image_bytes[pos + 6]
                          width = (image_bytes[pos + 7] << 8) | image_bytes[pos + 8]
                          return width, height
                      else:
                          print("Formato JPEG Inesperado, por favor, usar imagem com tamanho de 512x512.")
                          return None, None
              elif pos + 1 < size and image_bytes[pos + 1] == 0xD9:
                  break  # encontrou o marcador EOI fim da imagem
              elif pos + 1 < size and image_bytes[pos + 1] == 0xDA:
                  print("Atenção: Formato JPEG Progressivo detectado. A leitura das dimensões pode falhar.")
                  break
                #elif pos + 1 < size and image_bytes[pos + 1] == 0xEE:
                #  break
              pos += 1
            print("SOF0 marker não encontrado ou não está no formato esperado.")
            return None, None
        except Exception as e:
          print(f"Erro ao ler dimensões da imagem: {e}")
          return None, None

    def showImage(self):
        """Display the image."""
        if self.imageBytes is None:
            print("Erro: Imagem não carregada para exibição.")
            return
        try:
          from IPython.display import Image, display
          # Use IPython.display.Image to show the image.
          display(Image(data=self.imageBytes))
        except Exception as e:
            print(f"Erro ao exibir a imagem: {e}")


# Exemplo de uso:
image_path = '/content/drive/MyDrive/lena/lena.jpg'

file_manager = FileManagement(image_path)

pdb.set_trace()

width, height = file_manager.getImageDimensions()

print(file_manager.imageBytes)
print(file_manager.imageWithoutJPG)
print(f"Largura da imagem: {width}")
print(f"Altura da imagem: {height}")

if file_manager and file_manager.imageBytes: #garante que file_manager foi inicializado corretamente e que imageBytes existe
  file_manager.showImage()  # Mostra a imagem original
  image_instance = Image(file_manager)
  if image_instance.isImage(file_manager.imageBytes) and image_instance.width is not None and image_instance.height is not None:
    image_gray = ImageGrayScale(file_manager)
    if image_gray and image_gray.imageGray is not None:
       image_instance.printImagem(image_gray.imageGray, "Array em Tons de Cinza")
    image_binary = ImageBinary(file_manager, image_gray.imageGray)
    if image_binary and image_binary.imageBinary is not None:
        image_instance.printImagem(image_binary.imageBinary, "Array Binarizado")
    print("As imagens foram transformadas com sucesso.")
  else:
        print("Erro ao processar a imagem.")
else:
    print("Erro ao carregar a imagem.")

ValueError: mount failed